In [ ]:
import sys
import seaborn as sns
from matplotlib.colors import ListedColormap
from datetime import datetime, timedelta
import matplotlib.dates as mdates
import matplotlib.ticker as tkr
from matplotlib import rc
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import cmocean
import os
import warnings
import pandas as pd
import h5py
from datetime import datetime
from scipy import interpolate
sys.path.append('/Users/imarkelo/git/PorousMediaLab/')
from porousmedialab.batch import Batch

sns.set_style("whitegrid")
sns.set_style("ticks")
warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
sns.set_style("ticks")
rc('text', usetex=False)
rc("savefig", dpi=90)
rc("figure", dpi=90)
plt.rcParams['figure.figsize'] = 6, 4
pd.options.display.max_columns = 999
pd.options.display.max_rows = 400
str(datetime.now())
%matplotlib inline

In [ ]:
cl_stclaitr_209 = pd.read_excel('../../measurements/St Clair Lake/Chloride_in_Lake_St_Clair.xlsx', sheet_name='st. 209')[['Collect Date','Result B']]
cl_stclaitr_209['Collect Date'] = pd.to_datetime(cl_stclaitr_209['Collect Date'])
cl_stclaitr_209['Collect Date'] = pd.to_datetime(cl_stclaitr_209['Collect Date'])
cl_stclaitr_209['Date'] = cl_stclaitr_209['Collect Date'].dt.date
cl_stclaitr_209['Year'] = cl_stclaitr_209['Collect Date'].dt.year
cl_stclaitr_209['Month'] = cl_stclaitr_209['Collect Date'].dt.month
cl_stclaitr_209['seq_m'] = (cl_stclaitr_209['Year'] - 1996)*12 + cl_stclaitr_209['Month'] - shift

cl_stclaitr_210 = pd.read_excel('../../measurements/St Clair Lake/Chloride_in_Lake_St_Clair.xlsx', sheet_name='st. 210')[['Collect Date','Result B']]
cl_stclaitr_210['Collect Date'] = pd.to_datetime(cl_stclaitr_210['Collect Date'])
cl_stclaitr_210['Collect Date'] = pd.to_datetime(cl_stclaitr_210['Collect Date'])
cl_stclaitr_210['Date'] = cl_stclaitr_210['Collect Date'].dt.date
cl_stclaitr_210['Year'] = cl_stclaitr_210['Collect Date'].dt.year
cl_stclaitr_210['Month'] = cl_stclaitr_210['Collect Date'].dt.month
cl_stclaitr_210['seq_m'] = (cl_stclaitr_210['Year'] - 1996)*12 + cl_stclaitr_210['Month'] - shift

cl_stclaitr_210 = cl_stclaitr_210.groupby('seq_m').mean().reset_index()
cl_stclaitr_209 = cl_stclaitr_209.groupby('seq_m').mean().reset_index()

cl_stclaitr = pd.merge(cl_stclaitr_210, cl_stclaitr_209, on=['seq_m', 'Year'])[['seq_m','Year','Result B_y', 'Result B_x']]

cl_stclaitr['Cl'] = (cl_stclaitr['Result B_y'] + cl_stclaitr['Result B_x'])/2

cl_stclaitr = cl_stclaitr.groupby('seq_m').mean().reset_index()

In [ ]:
rivers_cl_kta = pd.read_csv('rivers_cl_kta_annual.csv')

In [ ]:
rivers = {'ca': {'clair': ['Thames_R', 'Sydenham_R', 'Ruscom_R'],
                 'eastern': ['Grand_R', 'Nanticoke_R', 'Lynn_R', 'Big_R'],
                 'central': ['Big_Otter_R', 'Kettle_R'],
                 'western': ['Turkey_R', 'Canard_R']},
          'us': {'clair': ['Clinton_R', 'Belle_R', 'Black_R_MI'],
                 'eastern': ['Cattaraugus_R', 'Buffalo_R'],
                 'central': ['Sandusky_R', 'Black_OH_R', 'Vermilion_R', 'Rocky_R', 'Cuyahoga_R',
                             'Chagrin_R', 'Grand_OH_R', 'Conneaut_R'],
                 'western': ['Rouge_R', 'Huron_MI_R', 'Raisin_R', 'Maumee_R', 'Portage_R']},
          'StClair': ['St_Clair_R']}
Lsc = rivers_cl_kta[
    ['Thames_R', 'Sydenham_R', 'Ruscom_R']+
    ['Clinton_R', 'Black_R_MI']].dropna().sum(axis=1)/0.75 + \
    rivers_cl_kta[['St_Clair_R']].dropna().sum(axis=1)
Lsc.plot()

In [ ]:
LTsc = Lsc.index.values
Lsc_spl = sp.interpolate.InterpolatedUnivariateSpline(LTsc, Lsc, ext=3)
plt.scatter(LTsc, Lsc)
plt.plot(LTsc, Lsc_spl(LTsc))

In [ ]:
discharge_ckmy = pd.read_csv('discharge_ckmy.csv').groupby(discharge_ckmy.level_0).mean()
Wsc = discharge_ckmy[rivers['us']['clair']+rivers['ca']['clair']].dropna().sum(axis=1)/0.75 + \
    discharge_ckmy[rivers['StClair']].dropna().sum(axis=1)
WTsc = Wsc.index.values
plt.plot(WTsc, Wsc)

In [ ]:
Wwb = discharge_ckmy[rivers['us']['western']+rivers['ca']['western']].sum(axis=1)/0.75
WTwb = Wwb.index.values
plt.plot(WTwb, Wwb)

In [ ]:
Lwb = rivers_cl_kta[rivers['us']['western']+rivers['ca']['western']].sum(axis=1)
LTwb = Lwb.index.values
Lwb_spl = sp.interpolate.InterpolatedUnivariateSpline(LTwb, Lwb, ext=3)
plt.scatter(LTwb, Lwb)
plt.plot(LTwb, Lwb_spl(LTwb))

In [ ]:
Wcb = discharge_ckmy[rivers['us']['central']+rivers['ca']['central']].sum(axis=1)/0.75
WTcb = Wcb.index.values
plt.plot(WTcb, Wcb)

In [ ]:
Lcb = rivers_cl_kta[rivers['us']['central']+rivers['ca']['central']].sum(axis=1)
LTcb = Lcb.index.values
Lcb_spl = sp.interpolate.InterpolatedUnivariateSpline(LTcb, Lcb, ext=3)
plt.scatter(LTcb, Lcb)
plt.plot(LTcb, Lcb_spl(LTcb))

In [ ]:
Web = discharge_ckmy[rivers['us']['eastern']+rivers['ca']['eastern']].sum(axis=1)/0.75
WTeb = Web.index.values
plt.plot(WTeb, Web)

In [ ]:
Leb = rivers_cl_kta[rivers['us']['central']+rivers['ca']['central']].sum(axis=1)
LTeb = Leb.index.values
Leb_spl = sp.interpolate.InterpolatedUnivariateSpline(LTeb, Leb, ext=3)
plt.scatter(LTeb, Leb)
plt.plot(LTeb, Leb_spl(LTeb))

In [ ]:
shift = 5

tend = 22 - shift
dt = 0.01
bl = Batch(tend, dt)
bl.add_species(name='Csc', init_conc=8.35)
bl.add_species(name='Cwb', init_conc=13)
bl.add_species(name='Ccb', init_conc=14)
bl.add_species(name='Ceb', init_conc=16)
bl.constants['Vsc'] = 4.17
bl.constants['Vwb'] = 25
bl.constants['Vcb'] = 305
bl.constants['Veb'] = 154

bl.constants['Dwc'] = 0.
bl.constants['Dce'] = 1.5

bl.functions['LTsc'] = np.array2string(Lsc.index.values-shift, separator=',')
bl.functions['Lsc'] = np.array2string(Lsc.values, separator=',')
bl.functions['WTsc'] = np.array2string(Wsc.index.values-shift, separator=',')
bl.functions['Wsc'] = np.array2string(Wsc.values, separator=',')
bl.functions['LTwb'] = np.array2string(Lwb.index.values-shift, separator=',')
bl.functions['Lwb'] = np.array2string(Lwb.values, separator=',')
bl.functions['WTwb'] = np.array2string(Wwb.index.values-shift, separator=',')
bl.functions['Wwb'] = np.array2string(Wwb.values, separator=',')
bl.functions['LTcb'] = np.array2string(Lcb.index.values-shift, separator=',')
bl.functions['Lcb'] = np.array2string(Lcb.values, separator=',')
bl.functions['WTcb'] = np.array2string(Wcb.index.values-shift, separator=',')
bl.functions['Wcb'] = np.array2string(Wcb.values, separator=',')
bl.functions['LTeb'] = np.array2string(Leb.index.values-shift, separator=',')
bl.functions['Leb'] = np.array2string(Leb.values, separator=',')
bl.functions['WTeb'] = np.array2string(Web.index.values-shift, separator=',')
bl.functions['Web'] = np.array2string(Web.values, separator=',')
bl.functions['Wsc_spl'] = 'sp.interpolate.InterpolatedUnivariateSpline(WTsc, Wsc,ext=3)'
bl.functions['Lsc_spl'] = 'sp.interpolate.InterpolatedUnivariateSpline(LTsc, Lsc, ext=3)'
bl.functions['Wwb_spl'] = 'sp.interpolate.InterpolatedUnivariateSpline(WTwb, Wwb,ext=3)'
bl.functions['Lwb_spl'] = 'sp.interpolate.InterpolatedUnivariateSpline(LTwb, Lwb, ext=3)'
bl.functions['Wcb_spl'] = 'sp.interpolate.InterpolatedUnivariateSpline(WTcb, Wcb,ext=3)'
bl.functions['Lcb_spl'] = 'sp.interpolate.InterpolatedUnivariateSpline(LTcb, Lcb, ext=3)'
bl.functions['Web_spl'] = 'sp.interpolate.InterpolatedUnivariateSpline(WTeb, Web,ext=3)'
bl.functions['Leb_spl'] = 'sp.interpolate.InterpolatedUnivariateSpline(LTeb, Leb, ext=3)'

bl.dcdt['Csc'] = 'Lsc_spl(TIME)/Vsc  - Wsc_spl(TIME)*Csc/Vsc'
bl.dcdt['Cwb'] = '(Lwb_spl(TIME)+300)/Vwb  + Wsc_spl(TIME)*Csc/Vwb - (Wsc_spl(TIME)+Wwb_spl(TIME))*Cwb/Vwb - Dwc*(Cwb - Ccb)'
bl.dcdt['Ccb'] = '(Lcb_spl(TIME)+150)/Vcb  + (Wsc_spl(TIME)+Wwb_spl(TIME))*Cwb/Vcb - (Wsc_spl(TIME)+Wwb_spl(TIME)+Wcb_spl(TIME))*Ccb/Vcb- Dwc*(Ccb - Cwb) - Dce*(Ccb - Ceb)'
bl.dcdt['Ceb'] = '(Leb_spl(TIME))/Veb  + (Wsc_spl(TIME)+Wwb_spl(TIME)+Wcb_spl(TIME))*Ccb/Veb - (Wsc_spl(TIME)+Wwb_spl(TIME)+Wcb_spl(TIME)+Web_spl(TIME))*Ceb/Veb - Dce*(Ceb - Ccb)'
bl.solve()

In [ ]:
f, axes = plt.subplots(1,1, figsize=(5,3), dpi=200, sharex=True)


axes.plot((bl.time + shift) + 1996, bl.species['Csc']['concentration'].flatten())
axes.scatter((cl_stclaitr['seq_m'].values+ shift-1)/12 + 1996, cl_stclaitr['Cl'].values, edgecolors=sns.color_palette()[3], s=50, marker='o', facecolors='white', zorder=10)
# axes.scatter((cl_stclaitr_210['seq_m'].values + shift)/12 + 1996, cl_stclaitr_210['Result B'].values, edgecolors=sns.color_palette()[3], s=50, marker='o', facecolors='white', zorder=10)

axes.set_ylim(0,14)
axes.set_xlim(2003,2017)
# axes.set_xlim(datetime(2002,12,31),datetime(2018,1,1))
# axes.legend(loc=2)
axes.set_ylabel('Lake St. Clair Cl, $mg\cdot L^{-1}$')


# handles, labels = axes.get_legend_handles_labels()
# f.legend( handles, labels, loc = (0.3, 0.83), ncol=2, frameon=1, framealpha=0.5, fontsize='small' )
# handles_2, labels_2 = ax2.get_legend_handles_labels()
# f.legend( handles + handles_2, labels+labels_2, loc = (0.18, 0.87), ncol=3, frameon=1, framealpha=0.5, fontsize='x-small' )
plt.tight_layout(h_pad=0.1)
axes.set_xlabel('')
axes.grid(lw=0.2)
plt.tight_layout()
# f.savefig('/Users/imarkelo/Google Drive/GDocuments/Lake Erie/Draft/Water Budget/latex_figures/img/evap_recent.pdf')






### Area weighted averages or depth integrated values:

In [ ]:
EB_A = pd.read_csv('Eastern_basin_averages_mgL.csv').groupby('Year').mean().reset_index()
CB_A = pd.read_csv('Central_basin_averages_mgL.csv').groupby('Year').mean().reset_index()
WB_A = pd.read_csv('Western_basin_averages_mgL.csv').groupby('Year').mean().reset_index()

In [ ]:
WB_AWA = pd.read_csv('Western_basin_area_weighted_averages_mgL.csv').groupby('Year').mean().reset_index()

f, axes = plt.subplots(1,1, figsize=(5,3), dpi=200, sharex=True)


axes.plot((bl.time + shift) + 1996, bl.species['Cwb']['concentration'].flatten())
# axes.scatter((cl_stclaitr['seq_m'].values+ shift)/12 + 1996, cl_stclaitr['Cl'].values, c=sns.color_palette()[1], s=10)
axes.scatter(WB_AWA.Year, WB_AWA.Value.values, edgecolors=sns.color_palette()[3], s=50, marker='o', facecolors='white', zorder=10)
# axes.scatter(WB_A.Year, WB_A['mean'].values, c=sns.color_palette()[3], s=50, marker='x', zorder=10)

# axes.set_ylim(0,None)
axes.set_xlim(2003,2017)
# axes.set_xlim(datetime(2002,12,31),datetime(2018,1,1))
# axes.legend(loc=2)
axes.set_ylabel('Lake Erie (Western) Cl, $mg\cdot L^{-1}$')


# handles, labels = axes.get_legend_handles_labels()
# f.legend( handles, labels, loc = (0.3, 0.83), ncol=2, frameon=1, framealpha=0.5, fontsize='small' )
# handles_2, labels_2 = ax2.get_legend_handles_labels()
# f.legend( handles + handles_2, labels+labels_2, loc = (0.18, 0.87), ncol=3, frameon=1, framealpha=0.5, fontsize='x-small' )
plt.tight_layout(h_pad=0.1)
axes.set_xlabel('')
axes.grid(lw=0.2)
plt.tight_layout()
# f.savefig('/Users/imarkelo/Google Drive/GDocuments/Lake Erie/Draft/Water Budget/latex_figures/img/evap_recent.pdf')






In [ ]:
CB_AWA = pd.read_csv('Central_basin_area_weighted_averages_mgL.csv').groupby('Year').mean().reset_index()

f, axes = plt.subplots(1,1, figsize=(5,3), dpi=200, sharex=True)


axes.plot((bl.time + shift) + 1996, bl.species['Ccb']['concentration'].flatten())
# axes.scatter((cl_stclaitr['seq_m'].values+ shift)/12 + 1996, cl_stclaitr['Cl'].values, c=sns.color_palette()[1], s=10)
axes.scatter(CB_AWA.Year + (CB_AWA.Month-1)/11, CB_AWA.Value.values, edgecolors=sns.color_palette()[3], s=50, marker='o', facecolors='white', zorder=10)
# axes.scatter(CB_A.Year + (CB_A.Month-1)/11, CB_A['mean'].values, c=sns.color_palette()[3], s=50, marker='x', zorder=10)

axes.set_ylim(0,None)
axes.set_xlim(2003,2017)
# axes.set_xlim(datetime(2002,12,31),datetime(2018,1,1))
# axes.legend(loc=2)
axes.set_ylabel('Lake Erie (Central) Cl, $mg\cdot L^{-1}$')


# handles, labels = axes.get_legend_handles_labels()
# f.legend( handles, labels, loc = (0.3, 0.83), ncol=2, frameon=1, framealpha=0.5, fontsize='small' )
# handles_2, labels_2 = ax2.get_legend_handles_labels()
# f.legend( handles + handles_2, labels+labels_2, loc = (0.18, 0.87), ncol=3, frameon=1, framealpha=0.5, fontsize='x-small' )
plt.tight_layout(h_pad=0.1)
axes.set_xlabel('')
axes.grid(lw=0.2)
plt.tight_layout()
# f.savefig('/Users/imarkelo/Google Drive/GDocuments/Lake Erie/Draft/Water Budget/latex_figures/img/evap_recent.pdf')






In [ ]:
EB_AWA = pd.read_csv('Eastern_basin_area_weighted_averages_mgL.csv').groupby('Year').mean().reset_index()

f, axes = plt.subplots(1,1, figsize=(5,3), dpi=200, sharex=True)


axes.plot((bl.time + shift) + 1996, bl.species['Ceb']['concentration'].flatten())
# axes.scatter((cl_stclaitr['seq_m'].values+ shift)/12 + 1996, cl_stclaitr['Cl'].values, c=sns.color_palette()[1], s=10)
axes.scatter(EB_AWA.Year, EB_AWA.Value.values, edgecolors=sns.color_palette()[3], s=50, marker='o', facecolors='white', zorder=10)
# axes.scatter(EB_A.Year , EB_A['mean'].values, c=sns.color_palette()[3], s=50, marker='x', zorder=10)

axes.set_ylim(0,None)
axes.set_xlim(2003,2017)
# axes.set_xlim(datetime(2002,12,31),datetime(2018,1,1))
# axes.legend(loc=2)
axes.set_ylabel('Lake Erie (Eastern) Cl, $mg\cdot L^{-1}$')


# handles, labels = axes.get_legend_handles_labels()
# f.legend( handles, labels, loc = (0.3, 0.83), ncol=2, frameon=1, framealpha=0.5, fontsize='small' )
# handles_2, labels_2 = ax2.get_legend_handles_labels()
# f.legend( handles + handles_2, labels+labels_2, loc = (0.18, 0.87), ncol=3, frameon=1, framealpha=0.5, fontsize='x-small' )
plt.tight_layout(h_pad=0.1)
axes.set_xlabel('')
axes.grid(lw=0.2)
plt.tight_layout()
# f.savefig('/Users/imarkelo/Google Drive/GDocuments/Lake Erie/Draft/Water Budget/latex_figures/img/evap_recent.pdf')






In [ ]:
EB_AWA = pd.read_csv('Eastern_basin_area_weighted_averages_mgL.csv')

f, axes = plt.subplots(1,1, figsize=(5,3), dpi=200, sharex=True)


axes.plot((bl.time + shift) + 1996, bl.species['Cwb']['concentration'].flatten(), label='WB')
axes.plot((bl.time + shift) + 1996, bl.species['Ccb']['concentration'].flatten(), label='CB')
axes.plot((bl.time + shift) + 1996, bl.species['Ceb']['concentration'].flatten(), label='EB')
# axes.scatter((cl_stclaitr['seq_m'].values+ shift)/12 + 1996, cl_stclaitr['Cl'].values, c=sns.color_palette()[1], s=10)

# axes.scatter(WB_AWA.Year + (WB_AWA.Month-1)/11, WB_AWA.Value.values, edgecolors=sns.color_palette()[0], s=30, marker='o', facecolors='white', zorder=10)
# axes.scatter(CB_AWA.Year + (CB_AWA.Month-1)/11, CB_AWA.Value.values, edgecolors=sns.color_palette()[1], s=30, marker='o', facecolors='white', zorder=10)
# axes.scatter(EB_AWA.Year + (EB_AWA.Month-1)/11, EB_AWA.Value.values, edgecolors=sns.color_palette()[2], s=30, marker='o', facecolors='white', zorder=10)


# axes.set_ylim(0,None)
axes.set_xlim(2003,2017)
# axes.set_xlim(datetime(2002,12,31),datetime(2018,1,1))
# axes.legend(loc=2)
axes.set_ylabel('Lake Erie (Eastern) Cl, $mg\cdot L^{-1}$')


handles, labels = axes.get_legend_handles_labels()
f.legend( handles, labels, loc = (0.3, 0.23), ncol=3, frameon=1, framealpha=0.5, fontsize='small' )
# handles_2, labels_2 = ax2.get_legend_handles_labels()
# f.legend( handles + handles_2, labels+labels_2, loc = (0.18, 0.87), ncol=3, frameon=1, framealpha=0.5, fontsize='x-small' )
plt.tight_layout(h_pad=0.1)
axes.set_xlabel('')
axes.grid(lw=0.2)
plt.tight_layout()
# f.savefig('/Users/imarkelo/Google Drive/GDocuments/Lake Erie/Draft/Water Budget/latex_figures/img/evap_recent.pdf')




